In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import copy
import time
import pprint
import shutil
from scipy.optimize import linprog
from pytest import approx
from IPython.display import HTML, display

# Descripción de la instancia:

Para realizar esta práctica, utilizamos una isntancia `m5.2xlarge`, que hemos ocupado en prácticas anteriores, tiene las siguientes características:

In [56]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:                        7
CPU MHz:                         3104.654
BogoMIPS:                        4999.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        35.8 MiB
NUMA node0 CPU(s):               

# Reimplementación 0

## Creamos archivos `.pyx`

In [2]:
os.chdir("..")
shutil.copyfile('src/mex/simplex/simplex_networks.py','notebooks/reimplementacion_0/simplex_networks_0.pyx')
shutil.copyfile('src/mex/simplex/problem_definition.py','notebooks/reimplementacion_0/problem_definition_0.pyx')
shutil.copyfile('src/mex/utils/general.py','notebooks/reimplementacion_0/general_0.pyx')
os.chdir("notebooks")

## Creamos archivos `setup.py`

In [10]:
%%file reimplementacion_0/setup_simplexnetworks_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/simplex_networks_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_simplexnetworks_0.py


In [11]:
%%file reimplementacion_0/setup_problemdefinition_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/problem_definition_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_problemdefinition_0.py


In [12]:
%%file reimplementacion_0/setup_general_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/general_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_general_0.py


## Compilando

In [13]:
%%bash
python3 reimplementacion_0/setup_simplexnetworks_0.py build_ext --inplace

running build_ext
building 'simplex_networks_0' extension
creating build
creating build/temp.linux-x86_64-3.8
creating build/temp.linux-x86_64-3.8/reimplementacion_0
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_0/simplex_networks_0.c -o build/temp.linux-x86_64-3.8/reimplementacion_0/simplex_networks_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_0/simplex_networks_0.o -o /home/ubuntu/practica-2-segunda-parte-caroacostatovany/notebooks/simplex_networks_0.c

In [14]:
%%bash
python3 reimplementacion_0/setup_problemdefinition_0.py build_ext --inplace

running build_ext
building 'problem_definition_0' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_0/problem_definition_0.c -o build/temp.linux-x86_64-3.8/reimplementacion_0/problem_definition_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_0/problem_definition_0.o -o /home/ubuntu/practica-2-segunda-parte-caroacostatovany/notebooks/problem_definition_0.cpython-38-x86_64-linux-gnu.so


In [15]:
%%bash
python3 reimplementacion_0/setup_general_0.py build_ext --inplace

running build_ext
building 'general_0' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_0/general_0.c -o build/temp.linux-x86_64-3.8/reimplementacion_0/general_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_0/general_0.o -o /home/ubuntu/practica-2-segunda-parte-caroacostatovany/notebooks/general_0.cpython-38-x86_64-linux-gnu.so


### Ejemplo pequeña escala

#### Valores Objetivo

In [16]:
c_max_obj_small = [-3, -5]
A_max_obj_small = [[1, 0], [0, 2], [3, 2]]
b_max_obj_small = [4, 12, 18]

In [17]:
max_obj_small = -1*linprog(c_max_obj_small, A_ub=A_max_obj_small, b_ub=b_max_obj_small).fun

In [18]:
coeff_obj_small = linprog(c_max_obj_small, A_ub=A_max_obj_small, b_ub=b_max_obj_small).x

#### Importamos módulos compilados y calculamos valores aproximados

In [19]:
import simplex_networks_0 as mex_0_sn
import problem_definition_0 as mex_0_pd

In [20]:
n_var_approx_small = 2
n_cons_approx_small = 3

In [21]:
# Todo junto
start_time = time.time()
matrix_max_approx_0_small = mex_0_sn.create_matrix(n_var_approx_small,n_cons_approx_small)
mex_0_pd.constrain(matrix_max_approx_0_small,'1,0,L,4')
mex_0_pd.constrain(matrix_max_approx_0_small,'0,2,L,12')
mex_0_pd.constrain(matrix_max_approx_0_small,'3,2,L,18')
mex_0_pd.obj(matrix_max_approx_0_small,'3,5,0')
problem_approx_0_small = mex_0_pd.maxz(matrix_max_approx_0_small)
max_approx_0_small = problem_approx_0_small['max']
problem_approx_0_small.pop('max')
coeff_approx_0_small = np.array(list(problem_approx_0_small.values()))
end_time = time.time()
secs_0_small = end_time-start_time
print("Todo el proceso tomó",secs_0_small,"segundos" )

Todo el proceso tomó 0.0005462169647216797 segundos


#### Comprobación

In [22]:
assert max_obj_small == approx(max_approx_0_small), "El valor aproximado es incorrecto"
assert coeff_obj_small == approx(coeff_approx_0_small), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_small)
print("El valor aproximado obtenido con mex es: ", max_approx_0_small)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_small)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_0_small)

El valor objetivo obtenido con scipy es:  35.99999997843401
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


### Ejemplo mediana escala

#### Valores Objetivo

In [23]:
c_max_obj_med = [-52.16, -45.51, -69.09, -84.88, -38.73,
              -84.47, -97.50, -61.32, -16.27, -36.42,
              -77.24, -36.91, -62.85, -50.77, -81.22,
              -66.94, -31.10, -45.05, -37.68, -40.76]

A_max_obj_med = [[90.09, 50.61, 45.03, 16.83, 26.92,
                36.29, 51.61, 2.450, 25.15, 30.22,
                81.78, 2.444, 34.71, 57.17, 41.14,
                92.00, 69.17, 26.77, 38.44, 25.35], 
               
               [69.62, 6.235, 18.95, 4.843, 12.34,
                82.56, 83.65, 11.18, 29.91, 34.66,
                79.32, 64.11, 95.56, 70.83, 69.77,
                93.99, 10.83, 17.88, 78.04, 87.34],
               
               [27.58, 31.56, 72.90, 95.74, 69.19,
                84.21, 58.42, 18.88, 72.27, 85.58,
                60.66, 51.78, 46.34, 30.96, 5.505,
                11.68, 84.84, 81.16, 3.668, 65.52],
               
               [28.91, 98.28, 80.47, 78.34, 48.34,
                40.00, 92.46, 93.44, 93.79, 46.63,
                50.50, 30.32, 71.96, 52.57, 46.29,
                67.03, 71.13, 82.94, 4.676, 5.110],
               
               [10.16, 9.828, 11.24, 53.81, 23.53,
                16.98, 94.65, 55.01, 96.43, 5.556,
                54.05, 40.95, 35.37, 14.02, 32.04,
                71.37, 29.52, 24.35, 71.32, 46.73],
               
               [70.56, 93.54, 27.93, 95.94, 91.23,
                8.491, 78.54, 7.435, 85.27, 32.27,
                97.65, 34.44, 97.45, 77.70, 86.22,
                3.460, 52.49, 59.86, 6.446, 20.77],
               
               [30.93, 40.11, 9.917, 22.34, 2.947,
                8.553, 85.74, 31.01, 4.919, 12.60,
                3.358, 89.39, 50.05, 25.59, 13.33,
                75.14, 57.81, 14.29, 88.59, 24.34],
               
               [7.511, 72.81, 19.49, 55.05, 86.04,
                36.84, 7.233, 67.59, 92.62, 24.06,
                15.87, 45.52, 75.98, 71.86, 64.45,
                15.81, 57.22, 84.86, 28.66, 7.867],
               
               [62.27, 38.88, 82.58, 84.71, 68.00,
                97.57, 36.58, 11.57, 73.91, 75.40,
                65.75, 7.540, 52.08, 96.61, 9.278,
                60.24, 59.22, 6.214, 63.88, 68.99],
               
               [73.05, 10.49, 93.23, 91.39, 63.19,
                47.53, 20.71, 14.26, 51.71, 71.02,
                11.87, 61.36, 97.72, 70.67, 28.40,
                73.55, 33.02, 81.46, 67.22, 85.30],
               
               [93.49, 28.98, 23.75, 48.18, 73.61,
                18.85, 15.89, 90.90, 71.75, 80.48,
                97.08, 88.23, 86.15, 57.57, 78.16,
                40.38, 19.41, 75.39, 54.97, 38.35],
               
               [84.61, 42.17, 56.76, 23.61, 66.47,
                13.86, 36.55, 37.61, 39.77, 2.123,
                92.07, 21.21, 85.36, 27.84, 81.87,
                34.29, 55.51, 71.08, 81.14, 41.18], 
               
               [66.07, 71.30, 29.57, 3.874, 45.04,
                98.19, 92.92, 18.87, 44.20, 17.36,
                36.33, 45.50, 17.97, 67.95, 62.03,
                32.55, 50.91, 47.08, 85.63, 36.12], 
               
               [63.66, 7.173, 29.93, 31.48, 64.60,
                72.68, 2.754, 16.71, 79.55, 4.050,
                95.43, 31.18, 61.87, 68.41, 3.567,
                21.61, 45.96, 3.881, 40.50, 91.86], 
               
               [67.60, 7.276, 78.37, 96.71, 79.11,
                21.35, 78.86, 23.26, 63.76, 43.93,
                59.46, 78.19, 70.34, 92.42, 58.98,
                95.50, 84.86, 76.11, 13.00, 60.67],
               
               [12.17, 45.38, 51.97, 93.29, 97.74,
                58.34, 71.99, 26.91, 26.85, 74.88,
                69.95, 43.51, 8.723, 38.97, 74.16,
                81.20, 77.52, 20.84, 46.12, 16.24],
               
               [85.82, 3.365, 53.87, 41.57, 37.04,
                97.64, 16.76, 90.16, 50.49, 76.92,
                85.17, 57.76, 29.29, 12.79, 93.86,
                59.50, 84.81, 43.91, 6.931, 14.25],
               
               [72.94, 37.67, 28.00, 33.56, 71.22,
                73.32, 33.33, 27.54, 71.75, 61.17,
                39.68, 38.85, 79.89, 92.34, 31.80,
                10.18, 78.30, 6.469, 41.89, 50.98],
               
               [80.03, 67.82, 37.00, 51.50, 17.85,
                10.91, 26.95, 65.79, 18.51, 84.34,
                3.196, 72.33, 48.76, 8.459, 53.13,
                18.23, 9.679, 2.543, 48.12, 52.09],
               
               [28.10, 98.61, 1.938, 94.54, 98.78,
                18.29, 26.88, 91.15, 9.967, 5.279,
                99.44, 79.94, 82.80, 30.05, 85.13,
                8.410, 83.47, 15.45, 46.43, 56.54],
               
               [92.44, 92.82, 95.87, 78.14, 93.72,
                55.82, 92.28, 64.95, 94.39, 91.63,
                20.11, 32.16, 10.08, 23.98, 9.257,
                71.39, 48.74, 60.76, 96.93, 56.57],
               
               [22.14, 72.51, 48.51, 98.13, 90.25,
                85.24, 1.804, 28.47, 45.93, 36.05,
                35.43, 7.104, 70.67, 97.69, 74.87,
                39.72, 13.98, 58.00, 2.024, 85.30],
               
               [72.16, 93.13, 45.85, 76.06, 22.53,
                36.16, 81.15, 92.51, 97.12, 67.37,
                37.37, 69.21, 72.32, 63.23, 92.90,
                61.86, 11.42, 34.15, 49.78, 6.688],
               
               [58.39, 34.12, 77.96, 5.433, 83.02,
                84.14, 59.82, 22.39, 70.75, 19.67,
                24.53, 41.00, 56.24, 89.32, 14.63,
                77.15, 34.12, 34.28, 55.17, 31.45],
               
               [20.15, 1.416, 74.34, 23.20, 8.632,
                71.47, 32.10, 59.23, 91.44, 37.58,
                23.29, 39.11, 19.54, 3.000, 2.319,
                43.50, 35.17, 66.78, 40.41, 50.82],
               
               [54.65, 70.46, 46.30, 66.92, 27.45,
                25.48, 81.67, 96.67, 42.57, 43.22,
                43.29, 60.58, 19.07, 27.73, 92.63,
                60.94, 84.33, 8.800, 78.62, 17.94],
               
               [83.10, 98.45, 22.09, 36.01, 39.99,
                6.380, 28.27, 10.69, 57.29, 63.19,
                44.42, 51.00, 15.57, 11.27, 63.56,
                17.54, 40.71, 47.53, 27.81, 27.66],
               
               [22.20, 29.98, 87.02, 25.26, 47.23,
                55.22, 83.70, 88.56, 67.82, 6.36,
                66.62, 3.853, 47.37, 97.22, 83.37,
                89.41, 61.79, 96.26, 91.11, 79.89],
               
               [6.828, 84.55, 80.55, 86.10, 54.30,
                18.86, 80.63, 54.07, 63.94, 89.28,
                64.89, 87.44, 13.06, 31.29, 63.10,
                75.33, 21.06, 61.33, 1.093, 91.08],
               
               [85.33, 57.98, 42.51, 53.95, 10.49,  
                32.74, 43.28, 51.52, 89.99, 43.97,
                46.69, 44.05, 85.91, 19.12, 23.85,
                58.82, 71.23, 8.885, 18.81, 87.44]]

b_max_obj_med = [79.72, 68.64, 1.240, 34.53, 43.64, 3.692, 44.21, 94.85, 39.16, 38.43,
               57.08, 19.63, 70.07, 32.45, 15.32, 49.46, 54.54, 24.96, 11.98, 1.772,
               50.61, 94.20, 97.49, 34.16, 6.26, 70.31, 79.18, 44.51, 53.22, 50.65]

In [24]:
max_obj_med = -1*linprog(c_max_obj_med, A_ub=A_max_obj_med, b_ub=b_max_obj_med).fun

In [25]:
coeff_obj_med = linprog(c_max_obj_med, A_ub=A_max_obj_med, b_ub=b_max_obj_med).x

#### Importamos módulos compilados y calculamos valores aproximados

In [26]:
import simplex_networks_0 as mex_0_sn
import problem_definition_0 as mex_0_pd

In [27]:
n_var_approx_med = 20
n_cons_approx_med = 30

In [28]:
# Todo junto
start_time = time.time()
matrix_max_approx_0_med = mex_0_sn.create_matrix(n_var_approx_med,n_cons_approx_med)
#Restricciones 1-5:
mex_0_pd.constrain(matrix_max_approx_0_med,'90.09,50.61,45.03,16.83,26.92,36.29,51.61,2.450,25.15,30.22,81.78,2.444,34.71,57.17,41.14,92.00,69.17,26.77,38.44,25.35,L,79.72')
mex_0_pd.constrain(matrix_max_approx_0_med,'69.62,6.235,18.95,4.843,12.34,82.56,83.65,11.18,29.91,34.66,79.32,64.11,95.56,70.83,69.77,93.99,10.83,17.88,78.04,87.34,L,68.64')
mex_0_pd.constrain(matrix_max_approx_0_med,'27.58,31.56,72.90,95.74,69.19,84.21,58.42,18.88,72.27,85.58,60.66,51.78,46.34,30.96,5.505,11.68,84.84,81.16,3.668,65.52,L,1.240')
mex_0_pd.constrain(matrix_max_approx_0_med,'28.91,98.28,80.47,78.34,48.34,40.00,92.46,93.44,93.79,46.63,50.50,30.32,71.96,52.57,46.29,67.03,71.13,82.94,4.676,5.110,L,34.53')
mex_0_pd.constrain(matrix_max_approx_0_med,'10.16,9.828,11.24,53.81,23.53,16.98,94.65,55.01,96.43,5.556,54.05,40.95,35.37,14.02,32.04,71.37,29.52,24.35,71.32,46.73,L,43.64')
#Restricciones 6-10:
mex_0_pd.constrain(matrix_max_approx_0_med,'70.56,93.54,27.93,95.94,91.23,8.491,78.54,7.435,85.27,32.27,97.65,34.44,97.45,77.70,86.22,3.460,52.49,59.86,6.446,20.77,L,3.692')
mex_0_pd.constrain(matrix_max_approx_0_med,'30.93,40.11,9.917,22.34,2.947,8.553,85.74,31.01,4.919,12.60,3.358,89.39,50.05,25.59,13.33,75.14,57.81,14.29,88.59,24.34,L,44.21')
mex_0_pd.constrain(matrix_max_approx_0_med,'7.511,72.81,19.49,55.05,86.04,36.84,7.233,67.59,92.62,24.06,15.87,45.52,75.98,71.86,64.45,15.81,57.22,84.86,28.66,7.867,L,94.85')
mex_0_pd.constrain(matrix_max_approx_0_med,'62.27,38.88,82.58,84.71,68.00,97.57,36.58,11.57,73.91,75.40,65.75,7.540,52.08,96.61,9.278,60.24,59.22,6.214,63.88,68.99,L,39.16')
mex_0_pd.constrain(matrix_max_approx_0_med,'73.05,10.49,93.23,91.39,63.19,47.53,20.71,14.26,51.71,71.02,11.87,61.36,97.72,70.67,28.40,73.55,33.02,81.46,67.22,85.30,L,38.43')
#Restricciones 11-15:
mex_0_pd.constrain(matrix_max_approx_0_med,'93.49,28.98,23.75,48.18,73.61,18.85,15.89,90.90,71.75,80.48,97.08,88.23,86.15,57.57,78.16,40.38,19.41,75.39,54.97,38.35,L,57.08')
mex_0_pd.constrain(matrix_max_approx_0_med,'84.61,42.17,56.76,23.61,66.47,13.86,36.55,37.61,39.77,2.123,92.07,21.21,85.36,27.84,81.87,34.29,55.51,71.08,81.14,41.18,L,19.63')
mex_0_pd.constrain(matrix_max_approx_0_med,'66.07,71.30,29.57,3.874,45.04,98.19,92.92,18.87,44.20,17.36,36.33,45.50,17.97,67.95,62.03,32.55,50.91,47.08,85.63,36.12,L,70.07')
mex_0_pd.constrain(matrix_max_approx_0_med,'63.66,7.173,29.93,31.48,64.60,72.68,2.754,16.71,79.55,4.050,95.43,31.18,61.87,68.41,3.567,21.61,45.96,3.881,40.50,91.86,L,32.45')
mex_0_pd.constrain(matrix_max_approx_0_med,'67.60,7.276,78.37,96.71,79.11,21.35,78.86,23.26,63.76,43.93,59.46,78.19,70.34,92.42,58.98,95.50,84.86,76.11,13.00,60.67,L,15.32')
#Restricciones 16-20:
mex_0_pd.constrain(matrix_max_approx_0_med,'12.17,45.38,51.97,93.29,97.74,58.34,71.99,26.91,26.85,74.88,69.95,43.51,8.723,38.97,74.16,81.20,77.52,20.84,46.12,16.24,L,49.46')
mex_0_pd.constrain(matrix_max_approx_0_med,'85.82,3.365,53.87,41.57,37.04,97.64,16.76,90.16,50.49,76.92,85.17,57.76,29.29,12.79,93.86,59.50,84.81,43.91,6.931,14.25,L,54.54')
mex_0_pd.constrain(matrix_max_approx_0_med,'72.94,37.67,28.00,33.56,71.22,73.32,33.33,27.54,71.75,61.17,39.68,38.85,79.89,92.34,31.80,10.18,78.30,6.469,41.89,50.98,L,24.96')
mex_0_pd.constrain(matrix_max_approx_0_med,'80.03,67.82,37.00,51.50,17.85,10.91,26.95,65.79,18.51,84.34,3.196,72.33,48.76,8.459,53.13,18.23,9.679,2.543,48.12,52.09,L,11.98')
mex_0_pd.constrain(matrix_max_approx_0_med,'28.10,98.61,1.938,94.54,98.78,18.29,26.88,91.15,9.967,5.279,99.44,79.94,82.80,30.05,85.13,8.410,83.47,15.45,46.43,56.54,L,1.772')
#Restricciones 21-25:
mex_0_pd.constrain(matrix_max_approx_0_med,'92.44,92.82,95.87,78.14,93.72,55.82,92.28,64.95,94.39,91.63,20.11,32.16,10.08,23.98,9.257,71.39,48.74,60.76,96.93,56.57,L,50.61')
mex_0_pd.constrain(matrix_max_approx_0_med,'22.14,72.51,48.51,98.13,90.25,85.24,1.804,28.47,45.93,36.05,35.43,7.104,70.67,97.69,74.87,39.72,13.98,58.00,2.024,85.30,L,94.20')
mex_0_pd.constrain(matrix_max_approx_0_med,'72.16,93.13,45.85,76.06,22.53,36.16,81.15,92.51,97.12,67.37,37.37,69.21,72.32,63.23,92.90,61.86,11.42,34.15,49.78,6.688,L,97.49')
mex_0_pd.constrain(matrix_max_approx_0_med,'58.39,34.12,77.96,5.433,83.02,84.14,59.82,22.39,70.75,19.67,24.53,41.00,56.24,89.32,14.63,77.15,34.12,34.28,55.17,31.45,L,34.16')
mex_0_pd.constrain(matrix_max_approx_0_med,'20.15,1.416,74.34,23.20,8.632,71.47,32.10,59.23,91.44,37.58,23.29,39.11,19.54,3.000,2.319,43.50,35.17,66.78,40.41,50.82,L,6.26')
#Restricciones 26-30:
mex_0_pd.constrain(matrix_max_approx_0_med,'54.65,70.46,46.30,66.92,27.45,25.48,81.67,96.67,42.57,43.22,43.29,60.58,19.07,27.73,92.63,60.94,84.33,8.800,78.62,17.94,L,70.31')
mex_0_pd.constrain(matrix_max_approx_0_med,'83.10,98.45,22.09,36.01,39.99,6.380,28.27,10.69,57.29,63.19,44.42,51.00,15.57,11.27,63.56,17.54,40.71,47.53,27.81,27.66,L,79.18')
mex_0_pd.constrain(matrix_max_approx_0_med,'22.20,29.98,87.02,25.26,47.23,55.22,83.70,88.56,67.82,6.36,66.62,3.853,47.37,97.22,83.37,89.41,61.79,96.26,91.11,79.89,L,44.51')
mex_0_pd.constrain(matrix_max_approx_0_med,'6.828,84.55,80.55,86.10,54.30,18.86,80.63,54.07,63.94,89.28,64.89,87.44,13.06,31.29,63.10,75.33,21.06,61.33,1.093,91.08,L,53.22')
mex_0_pd.constrain(matrix_max_approx_0_med,'85.33,57.98,42.51,53.95,10.49,32.74,43.28,51.52,89.99,43.97,46.69,44.05,85.91,19.12,23.85,58.82,71.23,8.885,18.81,87.44,L,50.65')
#Función objetivo:
mex_0_pd.obj(matrix_max_approx_0_med,'52.16,45.51,69.09,84.88,38.73,84.47,97.50,61.32,16.27,36.42,77.24,36.91,62.85,50.77,81.22,66.94,31.10,45.05,37.68,40.76,0')

problem_approx_0_med = mex_0_pd.maxz(matrix_max_approx_0_med)
max_approx_0_med = problem_approx_0_med['max']
problem_approx_0_med.pop('max')
coeff_approx_0_med = np.array(list(problem_approx_0_med.values()))
end_time = time.time()
secs_0_med = end_time-start_time
print("Todo el proceso tomó",secs_0_med,"segundos" )

Todo el proceso tomó 0.025148391723632812 segundos


#### Comprobación

In [29]:
assert max_obj_med == approx(max_approx_0_med), "El valor aproximado es incorrecto"
assert np.round(coeff_obj_med,3) == approx(coeff_approx_0_med, abs=1e-3), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_med)
print("El valor aproximado obtenido con mex es: ", max_approx_0_med)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_med)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_0_med)

El valor objetivo obtenido con scipy es:  7.644647466669042
El valor aproximado obtenido con mex es:  7.644647288586908
Los coeficientes objetivos obtenidos con scipy son:  [1.55767398e-10 2.13986131e-11 6.52653843e-11 2.03972271e-11
 2.20073205e-11 2.25421843e-11 2.95498944e-11 8.33451409e-11
 2.67709780e-11 2.57420289e-11 3.53108097e-11 2.45435519e-11
 3.71518342e-11 2.84650310e-10 1.08315701e-02 1.01059266e-01
 9.41232462e-12 3.40871440e-11 3.18768339e-10 2.14072463e-11]
Los coeficientes aproximados obtenidos con mex son:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.01083157 0.10105926 0.         0.
 0.         0.        ]


#### Tiempos

In [30]:
print("La iteración base a pequeña escala tardó", secs_0_small,"segundos")
print("La iteración base a mediana escala tardó", secs_0_med,"segundos")

La iteración base a pequeña escala tardó 0.0005462169647216797 segundos
La iteración base a mediana escala tardó 0.025148391723632812 segundos


## Anotaciones para analizar bloques de código

In [31]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/simplex_networks_0.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/problem_definition_0.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/general_0.pyx

In [32]:
display(HTML("reimplementacion_0/simplex_networks_0.html"))

In [33]:
display(HTML("reimplementacion_0/problem_definition_0.html"))

In [34]:
display(HTML("reimplementacion_0/general_0.html"))

# Reimplementación 1

De acuerdo a lo que podemos observar en las anotaciones previas así como los reportes del profiling, haremos cambios al código y volveremos a hacer la compilación a C.

## Creamos archivos `.pyx`

In [35]:
%%file reimplementacion_1/general_1.pyx

def convert_min(matrix):
    """
    This function multiplies by -1 the objective function for maximization problems. This is because
    if the problem to be solved is maximization then it is analogue to solve the problem -minimization.
    
    Args:
    
        matrix (numpy array): matrix to be updated.
    
    Returns:
    
        matrix (numpy array): matrix multiplied by -1.
    """
    
    cdef int i
    
    matrix[-1, :-2] = [-1*i for i in matrix[-1, :-2]]
    matrix[-1, -1] = -1*matrix[-1, -1]
    
    return matrix


def gen_var(matrix):
    """
    Generates the required number of variables. They are defined by the problem.
    
    Args:
    
        matrix (numpy array): matrix to be updated.
    
    Returns:
    
        v (list): list with problem variables.
    """
    
    cdef int i
    cdef long lc, lr, var
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    
    var = lc - lr -1
    v = []
    
    for i in range(var):
        v.append('x' + str(i+1))
    
    return v


def convert(eq):
    """
    Converts equation into a list containing the coefficients of the equation.
    
    Args:
    
        eq (string): equation defined with :func:`constrain`.
    
    Returns:
    
        eq (list): list with equation coefficients.
    """
    
    #cdef float i
    
    eq = eq.split(',')
    
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq
    
    if 'E' in eq:
        l = eq.index('E')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

Overwriting reimplementacion_1/general_1.pyx


In [36]:
%%file reimplementacion_1/simplex_networks_1.pyx

import numpy as np
import logging

def create_matrix(unsigned int variables, unsigned int constraints):
    """
    Creates a matrix with enough rows for each constraint plus the objective function
    and enough columns for all the variables.
    
    Args:
    
        variables (int): number of variables.
        
        constraints (int): number of constraints.
    
    Returns:
    
        matrix (numpy array): zero matrix.
    """
    
    matrix = np.zeros((constraints + 1, variables + constraints + 2))
    
    return matrix


def pivots_col(matrix):
    """
    Checks to see if pivots are required due to negative values in right column,
    excluding the bottom value.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        Flag (bool): True or False indicating whether a negative element was found.
    """
    
    cdef double m
    
    m = min(matrix[:-1, -1])
    if m >= 0:
        return False
    else:
        return True


def pivots_row(matrix):
    """
    Checks to see if pivots are required due to negative values in bottom row, 
    excluding the final value.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        Flag (bool): True or False indicating whether a negative element was found.
    """
    
    cdef int l
    cdef double m
    
    l = len(matrix[:, 0])
    m = min(matrix[l-1, :-1])
    if m >= 0:
        return False
    else:
        return True


def find_negative_col(matrix):
    """
    Finds location of negative values in right column.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        n (int): index of negative value in right column.
    """
    
    cdef int l
    cdef double m
    
    l = len(matrix[0, :])
    m = min(matrix[:-1, l-1])
    if m <= 0:
        n = np.where(matrix[:-1, l-1] == m)[0][0]
    else:
        n = None
    
    return n


def find_negative_row(matrix):
    """
    Finds location of negative values in bottom row.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        n (int): index of negative value in bottom row.
    """
    
    cdef int l
    cdef double m
    
    l = len(matrix[:,0])
    m = min(matrix[l-1,:-1])
    if m <= 0:
        n = np.where(matrix[l-1, :-1] == m)[0][0]
    else:
        n = None
    
    return n


def find_pivot_col(matrix):
    """
    Finds pivot element corresponding to a negative value in right column.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
        
        index (int): index of most negative value.
        
        c (int): index smallest value on row.
    """
    
    cdef int c, index
    cdef double m, i, j
    
    total = []
    neg = find_negative_col(matrix)
    row = matrix[neg, :-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = matrix[:-1, c]
    for i,j in zip(col, matrix[:-1,-1]):  #i for col with neg, j for right col
        if i != 0 and j/i > 0:
            total.append(j/i)
        else:
            total.append(10000) #placeholder, might need to update for large scale
    index = total.index(min(total))
    
    return [index,c]


def find_pivot_row(matrix):
    """
    Finds pivot element corresponding to a negative value in bottom row.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        index (int): index of most negative value.
        
        neg (int): index smallest value.
    """
    
    cdef double i, j
    cdef int index
    
    if pivots_row(matrix):
        total = []
        neg = find_negative_row(matrix)
        for i, j in zip(matrix[:-1, neg], matrix[:-1, -1]):
            if i != 0 and j/i > 0:
                total.append(j/i)
            else:
                total.append(10000) #placeholder, might need to update for large scale
        index = total.index(min(total))
        
        return [index,neg]


def pivot(row,col,matrix):
    """
    Pivot about a value to remove negative in final column or row.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
        row (int): position to pivot.
        
        col (int): position to pivot.
        
    Returns:
    
        t (numpy array): updated matrix.
    """
    
    cdef int i, lr, lc
    
    lr = len(matrix[:, 0])
    lc = len(matrix[0, :])
    t = np.zeros((lr, lc))
    pr = matrix[row, :]
    if matrix[row, col]**2 > 0:
        e = 1/matrix[row, col]
        r = pr*e
        for i in range(len(matrix[:, col])):
            k = matrix[i, :]
            c = matrix[i, col]
            if list(k) == list(pr):
                continue
            else:
                t[i, :] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        logging.info('Cannot pivot on this element')

Overwriting reimplementacion_1/simplex_networks_1.pyx


In [37]:
%%file reimplementacion_1/problem_definition_1.pyx

import numpy as np
import logging

from general_1 import gen_var, convert_min, convert
from simplex_networks_1 import pivots_col, find_pivot_col, pivots_row, pivot, find_pivot_row

def add_cons(matrix):
    """
    Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
    
    Returns:
    
        Flag (bool): True or False indicating whether 1+ constraints can be added.
    """

    cdef int i, j, total
    cdef long lr
    
    lr = len(matrix[:, 0])
    empty = []

    for i in range(lr):
        total = 0
        for j in matrix[i, :]:
            total += j**2
        if total == 0:
            empty.append(total)

    if len(empty) > 1:
        return True
    else:
        return False


def constrain(matrix, eq):
    """
    Adds constraints to the problem.
    
    Args:
    
        matrix (numpy array): matrix defined with :mod:`create_matrix`.
        
        eq (string): coefficients of constraints expressions. Use **L** for *less than*, **G** for *greater than*, **E** for *equal to*.
    
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'3,2,G,18')  # 3x_1 + 2x_2 >= 18
    """

    cdef int i, j, var, lc, lr
    
    if 'E' in eq:
        if add_cons(matrix):
            lc = len(matrix[0, :])
            lr = len(matrix[:, 0])
            var = lc - lr - 1
            j = 0

            while j < lr:
                row_check = matrix[j,:]
                total = 0
                for i in row_check:
                    total += float(i**2)
                if total == 0:
                    row = row_check
                    break
                j += 1
                
            eq = convert(eq)
            i = 0
            
            while i<len(eq)-1:
                row[i] = eq[i]
                i += 1
            
            row[-1] = eq[-1]
            #row[var+j] = 1
        
        else:
            logging.info('Cannot add another constraint.')
        
    else:
        if add_cons(matrix):
            lc = len(matrix[0, :])
            lr = len(matrix[:, 0])
            var = lc - lr -1
            j = 0
            
            while j < lr:
                row_check = matrix[j, :]
                total = 0
                for i in row_check:
                    total += float(i**2)
                if total == 0:
                    row = row_check
                    break
                j +=1
                
            eq = convert(eq)
            i = 0
            
            while i<len(eq)-1:
                row[i] = eq[i]
                i += 1
            
            row[-1] = eq[-1]
            row[var+j] = 1
            
        else:
            logging.info('Cannot add another constraint.')


def add_obj(matrix):
    """
    Verifies if the objective function can be added.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
    
    Returns:
    
        Flag (bool): True or False indicating whether objective function can be added.
    """
    
    cdef int i, j
    cdef long lr, total
    
    lr = len(matrix[:,0])
    empty = []
    
    for i in range(lr):
        total = 0
        for j in matrix[i, :]:
            total += j**2
        if total == 0:
            empty.append(total)
    
    if len(empty) == 1:
        return True
    else:
        return False


def obj(matrix,eq):
    """
    Adds the objective function to the problem matrix.
    
    .. note::
        Objective function must be added **after** constraints have been input.
    
    Args:
    
        matrix (numpy array): matrix defined with :mod:`create_matrix`.
        
        eq (string): coefficients of objective function.
    
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'4,2,G,18')  # 4x_1 + 2x_2 >= 18
    >>> obj(problem_matrix,'3,5,0')           # 3x_1 + 5x_2
    """
    
    cdef long lr
    
    if add_obj(matrix):
        eq = [float(i) for i in eq.split(',')]
        lr = len(matrix[:,0])
        row = matrix[lr-1,:]
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        logging.info('You must finish adding constraints before the objective function can be added.')


def maxz(matrix):
    """
    Creates maximization function. Determines if 1 extra pivot is required, locates the pivot element,
    pivots about it and continues the process until all negative elements have been removed from
    the last column and row.
    
    Args:
    
        matrix (numpy array): problem matrix with constraints and objective function added.
    
    Returns:
    
        *(dict)* A dictionary with Max and variables.
        
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'4,2,G,18')  # 4x_1 + 2x_2 >= 18
    >>> obj(problem_matrix,'3,5,0')           # 3x_1 + 5x_2
    >>> maxz(problem_matrix)
    {'x1': 4.0, 'x2': 6.0, 'max': 42.0}
    """
    
    cdef int i
    cdef float s, m
    cdef long lc, lr, var
    
    while pivots_col(matrix):
        matrix = pivot(find_pivot_col(matrix)[0], find_pivot_col(matrix)[1], matrix)
    while pivots_row(matrix):
        matrix = pivot(find_pivot_row(matrix)[0], find_pivot_row(matrix)[1], matrix)
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    
    for i in range(var):
        col = matrix[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(matrix)[i]] = matrix[loc, -1]
        else:
            val[gen_var(matrix)[i]] = 0
    val['max'] = matrix[-1, -1]
    
    return val


def minz(matrix):
    """
    Creates minimization function. Determines if 1 extra pivot is required, locates the pivot element,
    pivots about it and continues the process until all negative elements have been removed from 
    the last column and row.
    
    Args:
    
        matrix (numpy array): problem matrix with constraints and objective function added.
    
    Returns:
    
        (dict) A dictionary with Min and variables.
        
    >>> problem_matrix = create_matrix(2,4)   # 2 variables and 4 constraints
    >>> constrain(problem_matrix,'1,1,L,6')   # x_1 + x_2 <= 6
    >>> constrain(problem_matrix,'-1,2,L,8')  # -x_1 + 2x_2 <= 8
    >>> constrain(problem_matrix,'1,G,0')     # x_1 >= 0
    >>> constrain(problem_matrix,'0,1,G,0')   # x_2 >= 0
    >>> obj(problem_matrix,'-1,-3,0')         # -x_1 - 3x_2
    >>> minz(problem_matrix)
    {'x1': 1.3333333333333333, 'x2': 4.666666666666667}
    """
    
    cdef int i
    cdef float s, m
    cdef long lc, lr, var

    matrix = convert_min(matrix)
    while pivots_col(matrix):
        matrix = pivot(find_pivot_col(matrix)[0], find_pivot_col(matrix)[1], matrix)
    while pivots_row(matrix):
        matrix = pivot(find_pivot_row(matrix)[0], find_pivot_row(matrix)[1], matrix)
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    
    for i in range(var):
        col = matrix[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(matrix)[i]] = matrix[loc, -1]
        else:
            val[gen_var(matrix)[i]] = 0
    val['min'] = matrix[-1, -1]*-1
    
    return val


Overwriting reimplementacion_1/problem_definition_1.pyx


## Creamos archivos `setup.py`

In [38]:
%%file reimplementacion_1/setup_simplexnetworks_1.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_1/simplex_networks_1.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_1/setup_simplexnetworks_1.py


In [39]:
%%file reimplementacion_1/setup_problemdefinition_1.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_1/problem_definition_1.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_1/setup_problemdefinition_1.py


In [40]:
%%file reimplementacion_1/setup_general_1.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_1/general_1.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_1/setup_general_1.py


## Compilando

In [41]:
%%bash
python3 reimplementacion_1/setup_simplexnetworks_1.py build_ext --inplace

Compiling reimplementacion_1/simplex_networks_1.pyx because it changed.
[1/1] Cythonizing reimplementacion_1/simplex_networks_1.pyx
running build_ext
building 'simplex_networks_1' extension
creating build/temp.linux-x86_64-3.8/reimplementacion_1
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_1/simplex_networks_1.c -o build/temp.linux-x86_64-3.8/reimplementacion_1/simplex_networks_1.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_1/simplex_networks_1.o -o /home

In [42]:
%%bash
python3 reimplementacion_1/setup_problemdefinition_1.py build_ext --inplace

Compiling reimplementacion_1/problem_definition_1.pyx because it changed.
[1/1] Cythonizing reimplementacion_1/problem_definition_1.pyx
running build_ext
building 'problem_definition_1' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_1/problem_definition_1.c -o build/temp.linux-x86_64-3.8/reimplementacion_1/problem_definition_1.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_1/problem_definition_1.o -o /home/ubuntu/practica-2-segunda-parte-caroacostat

In [43]:
%%bash
python3 reimplementacion_1/setup_general_1.py build_ext --inplace

Compiling reimplementacion_1/general_1.pyx because it changed.
[1/1] Cythonizing reimplementacion_1/general_1.pyx
running build_ext
building 'general_1' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c reimplementacion_1/general_1.c -o build/temp.linux-x86_64-3.8/reimplementacion_1/general_1.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/reimplementacion_1/general_1.o -o /home/ubuntu/practica-2-segunda-parte-caroacostatovany/notebooks/general_1.cpython-38-x86_64-linux-gnu.so


### Ejemplo pequeña escala

#### Importamos módulos compilados y calculamos valores aproximados

In [44]:
import simplex_networks_1 as mex_1_sn
import problem_definition_1 as mex_1_pd

In [45]:
# Todo junto
start_time = time.time()
matrix_max_approx_1_small = mex_1_sn.create_matrix(n_var_approx_small,n_cons_approx_small)
mex_1_pd.constrain(matrix_max_approx_1_small,'1,0,L,4')
mex_1_pd.constrain(matrix_max_approx_1_small,'0,2,L,12')
mex_1_pd.constrain(matrix_max_approx_1_small,'3,2,L,18')
mex_1_pd.obj(matrix_max_approx_1_small,'3,5,0')
problem_approx_1_small = mex_1_pd.maxz(matrix_max_approx_1_small)
max_approx_1_small = problem_approx_1_small['max']
problem_approx_1_small.pop('max')
coeff_approx_1_small = np.array(list(problem_approx_1_small.values()))
end_time = time.time()
secs_1_small = end_time-start_time
print("Todo el proceso tomó",secs_1_small,"segundos" )

Todo el proceso tomó 0.0005617141723632812 segundos


#### Comprobación

In [46]:
assert max_obj_small == approx(max_approx_1_small), "El valor aproximado es incorrecto"
assert coeff_obj_small == approx(coeff_approx_1_small), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_small)
print("El valor aproximado obtenido con mex es: ", max_approx_1_small)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_small)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1_small)

El valor objetivo obtenido con scipy es:  35.99999997843401
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


### Ejemplo mediana escala

#### Importamos módulos compilados y calculamos valores aproximados

In [47]:
import simplex_networks_1 as mex_1_sn
import problem_definition_1 as mex_1_pd

In [48]:
# Todo junto
start_time = time.time()
matrix_max_approx_1_med = mex_1_sn.create_matrix(n_var_approx_med,n_cons_approx_med)
#Restricciones 1-5:
mex_1_pd.constrain(matrix_max_approx_1_med,'90.09,50.61,45.03,16.83,26.92,36.29,51.61,2.450,25.15,30.22,81.78,2.444,34.71,57.17,41.14,92.00,69.17,26.77,38.44,25.35,L,79.72')
mex_1_pd.constrain(matrix_max_approx_1_med,'69.62,6.235,18.95,4.843,12.34,82.56,83.65,11.18,29.91,34.66,79.32,64.11,95.56,70.83,69.77,93.99,10.83,17.88,78.04,87.34,L,68.64')
mex_1_pd.constrain(matrix_max_approx_1_med,'27.58,31.56,72.90,95.74,69.19,84.21,58.42,18.88,72.27,85.58,60.66,51.78,46.34,30.96,5.505,11.68,84.84,81.16,3.668,65.52,L,1.240')
mex_1_pd.constrain(matrix_max_approx_1_med,'28.91,98.28,80.47,78.34,48.34,40.00,92.46,93.44,93.79,46.63,50.50,30.32,71.96,52.57,46.29,67.03,71.13,82.94,4.676,5.110,L,34.53')
mex_1_pd.constrain(matrix_max_approx_1_med,'10.16,9.828,11.24,53.81,23.53,16.98,94.65,55.01,96.43,5.556,54.05,40.95,35.37,14.02,32.04,71.37,29.52,24.35,71.32,46.73,L,43.64')
#Restricciones 6-10:
mex_1_pd.constrain(matrix_max_approx_1_med,'70.56,93.54,27.93,95.94,91.23,8.491,78.54,7.435,85.27,32.27,97.65,34.44,97.45,77.70,86.22,3.460,52.49,59.86,6.446,20.77,L,3.692')
mex_1_pd.constrain(matrix_max_approx_1_med,'30.93,40.11,9.917,22.34,2.947,8.553,85.74,31.01,4.919,12.60,3.358,89.39,50.05,25.59,13.33,75.14,57.81,14.29,88.59,24.34,L,44.21')
mex_1_pd.constrain(matrix_max_approx_1_med,'7.511,72.81,19.49,55.05,86.04,36.84,7.233,67.59,92.62,24.06,15.87,45.52,75.98,71.86,64.45,15.81,57.22,84.86,28.66,7.867,L,94.85')
mex_1_pd.constrain(matrix_max_approx_1_med,'62.27,38.88,82.58,84.71,68.00,97.57,36.58,11.57,73.91,75.40,65.75,7.540,52.08,96.61,9.278,60.24,59.22,6.214,63.88,68.99,L,39.16')
mex_1_pd.constrain(matrix_max_approx_1_med,'73.05,10.49,93.23,91.39,63.19,47.53,20.71,14.26,51.71,71.02,11.87,61.36,97.72,70.67,28.40,73.55,33.02,81.46,67.22,85.30,L,38.43')
#Restricciones 11-15:
mex_1_pd.constrain(matrix_max_approx_1_med,'93.49,28.98,23.75,48.18,73.61,18.85,15.89,90.90,71.75,80.48,97.08,88.23,86.15,57.57,78.16,40.38,19.41,75.39,54.97,38.35,L,57.08')
mex_1_pd.constrain(matrix_max_approx_1_med,'84.61,42.17,56.76,23.61,66.47,13.86,36.55,37.61,39.77,2.123,92.07,21.21,85.36,27.84,81.87,34.29,55.51,71.08,81.14,41.18,L,19.63')
mex_1_pd.constrain(matrix_max_approx_1_med,'66.07,71.30,29.57,3.874,45.04,98.19,92.92,18.87,44.20,17.36,36.33,45.50,17.97,67.95,62.03,32.55,50.91,47.08,85.63,36.12,L,70.07')
mex_1_pd.constrain(matrix_max_approx_1_med,'63.66,7.173,29.93,31.48,64.60,72.68,2.754,16.71,79.55,4.050,95.43,31.18,61.87,68.41,3.567,21.61,45.96,3.881,40.50,91.86,L,32.45')
mex_1_pd.constrain(matrix_max_approx_1_med,'67.60,7.276,78.37,96.71,79.11,21.35,78.86,23.26,63.76,43.93,59.46,78.19,70.34,92.42,58.98,95.50,84.86,76.11,13.00,60.67,L,15.32')
#Restricciones 16-20:
mex_1_pd.constrain(matrix_max_approx_1_med,'12.17,45.38,51.97,93.29,97.74,58.34,71.99,26.91,26.85,74.88,69.95,43.51,8.723,38.97,74.16,81.20,77.52,20.84,46.12,16.24,L,49.46')
mex_1_pd.constrain(matrix_max_approx_1_med,'85.82,3.365,53.87,41.57,37.04,97.64,16.76,90.16,50.49,76.92,85.17,57.76,29.29,12.79,93.86,59.50,84.81,43.91,6.931,14.25,L,54.54')
mex_1_pd.constrain(matrix_max_approx_1_med,'72.94,37.67,28.00,33.56,71.22,73.32,33.33,27.54,71.75,61.17,39.68,38.85,79.89,92.34,31.80,10.18,78.30,6.469,41.89,50.98,L,24.96')
mex_1_pd.constrain(matrix_max_approx_1_med,'80.03,67.82,37.00,51.50,17.85,10.91,26.95,65.79,18.51,84.34,3.196,72.33,48.76,8.459,53.13,18.23,9.679,2.543,48.12,52.09,L,11.98')
mex_1_pd.constrain(matrix_max_approx_1_med,'28.10,98.61,1.938,94.54,98.78,18.29,26.88,91.15,9.967,5.279,99.44,79.94,82.80,30.05,85.13,8.410,83.47,15.45,46.43,56.54,L,1.772')
#Restricciones 21-25:
mex_1_pd.constrain(matrix_max_approx_1_med,'92.44,92.82,95.87,78.14,93.72,55.82,92.28,64.95,94.39,91.63,20.11,32.16,10.08,23.98,9.257,71.39,48.74,60.76,96.93,56.57,L,50.61')
mex_1_pd.constrain(matrix_max_approx_1_med,'22.14,72.51,48.51,98.13,90.25,85.24,1.804,28.47,45.93,36.05,35.43,7.104,70.67,97.69,74.87,39.72,13.98,58.00,2.024,85.30,L,94.20')
mex_1_pd.constrain(matrix_max_approx_1_med,'72.16,93.13,45.85,76.06,22.53,36.16,81.15,92.51,97.12,67.37,37.37,69.21,72.32,63.23,92.90,61.86,11.42,34.15,49.78,6.688,L,97.49')
mex_1_pd.constrain(matrix_max_approx_1_med,'58.39,34.12,77.96,5.433,83.02,84.14,59.82,22.39,70.75,19.67,24.53,41.00,56.24,89.32,14.63,77.15,34.12,34.28,55.17,31.45,L,34.16')
mex_1_pd.constrain(matrix_max_approx_1_med,'20.15,1.416,74.34,23.20,8.632,71.47,32.10,59.23,91.44,37.58,23.29,39.11,19.54,3.000,2.319,43.50,35.17,66.78,40.41,50.82,L,6.26')
#Restricciones 26-30:
mex_1_pd.constrain(matrix_max_approx_1_med,'54.65,70.46,46.30,66.92,27.45,25.48,81.67,96.67,42.57,43.22,43.29,60.58,19.07,27.73,92.63,60.94,84.33,8.800,78.62,17.94,L,70.31')
mex_1_pd.constrain(matrix_max_approx_1_med,'83.10,98.45,22.09,36.01,39.99,6.380,28.27,10.69,57.29,63.19,44.42,51.00,15.57,11.27,63.56,17.54,40.71,47.53,27.81,27.66,L,79.18')
mex_1_pd.constrain(matrix_max_approx_1_med,'22.20,29.98,87.02,25.26,47.23,55.22,83.70,88.56,67.82,6.36,66.62,3.853,47.37,97.22,83.37,89.41,61.79,96.26,91.11,79.89,L,44.51')
mex_1_pd.constrain(matrix_max_approx_1_med,'6.828,84.55,80.55,86.10,54.30,18.86,80.63,54.07,63.94,89.28,64.89,87.44,13.06,31.29,63.10,75.33,21.06,61.33,1.093,91.08,L,53.22')
mex_1_pd.constrain(matrix_max_approx_1_med,'85.33,57.98,42.51,53.95,10.49,32.74,43.28,51.52,89.99,43.97,46.69,44.05,85.91,19.12,23.85,58.82,71.23,8.885,18.81,87.44,L,50.65')
#Función objetivo:
mex_1_pd.obj(matrix_max_approx_1_med,'52.16,45.51,69.09,84.88,38.73,84.47,97.50,61.32,16.27,36.42,77.24,36.91,62.85,50.77,81.22,66.94,31.10,45.05,37.68,40.76,0')

problem_approx_1_med = mex_1_pd.maxz(matrix_max_approx_1_med)
max_approx_1_med = problem_approx_1_med['max']
problem_approx_1_med.pop('max')
coeff_approx_1_med = np.array(list(problem_approx_1_med.values()))
end_time = time.time()
secs_1_med = end_time-start_time
print("Todo el proceso tomó",secs_1_med,"segundos" )

Todo el proceso tomó 0.009022951126098633 segundos


#### Comprobación

In [49]:
assert max_obj_med == approx(max_approx_1_med), "El valor aproximado es incorrecto"
assert np.round(coeff_obj_med,3) == approx(coeff_approx_1_med, abs=1e-3), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_med)
print("El valor aproximado obtenido con mex es: ", max_approx_1_med)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_med)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1_med)

El valor objetivo obtenido con scipy es:  7.644647466669042
El valor aproximado obtenido con mex es:  7.644647288586908
Los coeficientes objetivos obtenidos con scipy son:  [1.55767398e-10 2.13986131e-11 6.52653843e-11 2.03972271e-11
 2.20073205e-11 2.25421843e-11 2.95498944e-11 8.33451409e-11
 2.67709780e-11 2.57420289e-11 3.53108097e-11 2.45435519e-11
 3.71518342e-11 2.84650310e-10 1.08315701e-02 1.01059266e-01
 9.41232462e-12 3.40871440e-11 3.18768339e-10 2.14072463e-11]
Los coeficientes aproximados obtenidos con mex son:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.01083157 0.10105926 0.         0.
 0.         0.        ]


#### Tiempos

In [50]:
print("La iteración 1 a pequeña escala tardó", secs_1_small,"segundos")
print("La iteración 1 a mediana escala tardó", secs_1_med,"segundos")

La iteración 1 a pequeña escala tardó 0.0005617141723632812 segundos
La iteración 1 a mediana escala tardó 0.009022951126098633 segundos


In [51]:
print("De la iteración base a la iteración 1 en pequeña escala se redujeron", secs_0_small-secs_1_small,"segundos")
print("De la iteración base a la iteración 1 en mediana escala se redujeron", secs_0_med-secs_1_med,"segundos")

De la iteración base a la iteración 1 en pequeña escala se redujeron -1.5497207641601562e-05 segundos
De la iteración base a la iteración 1 en mediana escala se redujeron 0.01612544059753418 segundos


## Anotaciones para analizar bloques de código

In [52]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_1/simplex_networks_1.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_1/problem_definition_1.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_1/general_1.pyx

In [53]:
display(HTML("reimplementacion_1/simplex_networks_1.html"))

In [54]:
display(HTML("reimplementacion_1/problem_definition_1.html"))

In [55]:
display(HTML("reimplementacion_1/general_1.html"))